# The StatQuest Illustrated Guide to Neural Networks and AI
## Chapter 6 - Convolutional Neural Networks!!!

Copyright 2024, Joshua Starmer

---- 

This tutorial is from the book, **[The StatQuest Illustrated Guide to Neural Networks and AI](https://www.amazon.com/dp/B0DRS71QVQ)**.

In this notebook, we will use **[PyTorch](https://pytorch.org/) + [Lightning](https://www.lightning.ai/)** to create and optimize a simple Convolutional Neural Network (CNN) like the one shown in the picture below. Convolutional neural networks are used for image classification, so when your phone thinks you've taken a picture of a cute kitten, that picture was probably classified with a CNN. In this case, we will create a network that can classify Xs and Os during a game of tic-tac-toe.

<!-- <img src="./images/annotated_cnn.png" alt="a simple convolutional neural network" style="width: 600px;"> -->
<img src="./images/full_cnn.png" alt="a simple convolutional neural network" style="width: 800px;">

In this tutorial, you will...

- **[Create Data and then Build a DataLoader From Scratch](#data)**
- **[Build a Simple Convolutional Neural Network](#build)**
- **[Train the Convolutional Neural Network](#train)**
- **[Classify New Images](#predict)**

#### NOTE:
This tutorial assumes that you already know the basics of coding in **Python** and are familiar with the theory behind **[Convolutional Neural Networks](https://youtu.be/HGwBXDKFk9I)** and  **[Backpropagation](https://youtu.be/IN2XmBhILt4)**. If not, check out the **StatQuests** by clicking on the links for each topic.

-----

# Import the modules that will do all the work

The very first thing we need to do is load a bunch of Python modules. Python itself is just a basic programming language. These modules give us extra functionality to create and train a Neural Network.

In [ ]:
%%capture 
# %%capture prevents this cell from printing a ton of STDERR stuff to the screen

## First, check to see if lightning is installed, if not, install it.
##
## NOTE: If you **do** need to install something, just know that you may need to
##       restart your session for python to find the new module(s).
##
##       To restart your session:
##       - In Google Colab, click on the "Runtime" menu and select
##         "Restart Session" from the pulldown menu
##       - In a local jupyter notebook, click on the "Kernel" menu and select
##         "Restart Kernel" from the pulldown menu
import pip
try:
  __import__("lightning")
except ImportError:
  pip.main(['install', "lightning"])  

In [ ]:
import torch # torch will allow us to create tensors.
import torch.nn as nn # torch.nn allows us to create a neural network.
import torch.nn.functional as F # nn.functional give us access to the activation and loss functions.
from torch.optim import Adam # optim contains many optimizers. This time we're using Adam

import lightning as L # lightning has tons of cool tools that make neural networks easier
from torch.utils.data import TensorDataset, DataLoader # these are needed for the training data

import matplotlib.pyplot as plt ## matplotlib allows us to draw the images used for input.

## NOTE: If you get an error running this block of code, it is probably
##       because you installed a new package earlier and forgot to
##       restart your session for python to find the new module(s).
##
##       To restart your session:
##       - In Google Colab, click on the "Runtime" menu and select
##         "Restart Session" from the pulldown menu
##       - In a local jupyter notebook, click on the "Kernel" menu and select
##         "Restart Kernel" from the pulldown menu

----

<a id="data"></a>
# Create Images and then Build a DataLoader From Scratch.

Once we have the Python modules imported, we need to create the images to train and test our neural network. Specifically, we'll create an image of the letters **O** and **X** that look like the pictures below.

<img src="./images/training_data.png" alt="a convolutional neural network" style="width: 800px;">

We'll start by creating the image of the letter **O** by creating a 6x6 matrix of numbers where 0 represents white and 1 represents black.

In [ ]:
## Create a 6x6 matrix of numbers where 0 represents white
## and 1 represents black.
o_image = [[0, 0, 1, 1, 0, 0],
           [0, 1, 0, 0, 1, 0],
           [1, 0, 0, 0, 0, 1],
           [1, 0, 0, 0, 0, 1],
           [0, 1, 0, 0, 1, 0],
           [0, 0, 1, 1, 0, 0]]
o_image # print out the matrix to verify that it is what we expect

Now, let's create an image of the letter **X** by creating a similar 6x6 matrix, where the 1s are now in an **X** pattern.

In [ ]:
x_image = [[1, 0, 0, 0, 0, 1],
           [0, 1, 0, 0, 1, 0],
           [0, 0, 1, 1, 0, 0],
           [0, 0, 1, 1, 0, 0],
           [0, 1, 0, 0, 1, 0],
           [1, 0, 0, 0, 0, 1]]
x_image

By squinting at the matrices we created for `o_image` and `x_image`, we can sort of see that we made them correctly. However, by drawing the images with matplotlib, we can get a much clearer picture, or rather, we can get much clearer pictures, of what we have done so far.

In [ ]:
## To draw the o_image and x_image, we first call subplots(), which creates 
## an array, called axarr[], with an entry for each element in a grid
## specified by nrows and ncols.
fig, axarr = plt.subplots(nrows=1, ncols=2, figsize=(5, 5))

## Now we pass o_image and x_image to .imshow() for each element
## in the grid created by plt.subplots()
axarr[0].imshow(o_image, cmap='gray_r') ## Setting cmap='gray_r' gives us reverse grayscale.
axarr[1].imshow(x_image, cmap='gray_r')

Now, let's put our training data into a **DataLoader**, which we can use to train the neural network. **DataLoaders** are great for large datasets because they make it easy to access the data in batches, make it easy to shuffle the data each epoch, and they make it easy to use a relatively small fraction of the data if we want to do a quick and dirty training for debugging our code.

To put our data training data into a **DataLoader**...

- We'll start by converting the images into tensors with `torch.tensor()` and save them as `input_images`. 

In [ ]:
## Convert the images into tensors...
input_images = torch.tensor([o_image, x_image]).type(torch.float32)

**NOTE:** When we call `torch.tensor()`, we tack on `type(torch.float32)` to ensure the numbers are saved in the correct format for the neural network to process efficiently.

- Then, we'll create tensors for the labels, the ideal output values given each input image. In this example, our convolutional neural network has 2 outputs, one for the letter **O** and one for the letter **X**, so `[1.0, 0.0]` will represent the ideal output for the letter **O**, and `[0.0, 1.0]` will represent the ideal output for the letter **X**. The labels will be saved in `input_labels`.

In [ ]:
## Create the labels for the input images
input_labels = torch.tensor([[1.0, 0.0], [0.0, 1.0]]).type(torch.float32)

- Then, we'll combine `input_images` with `input_labels` to create a **TensorDataset** and use the **TensorDataset** to create the **DataLoader**.

In [ ]:
## Now combine input_images and input_labels into a TensorDataset...
dataset = TensorDataset(input_images, input_labels) 
## ...and use the TensorDataset to create a DataLoader.
dataloader = DataLoader(dataset)

Now, just for fun, we can verify that `dataloader` contains the input images and labels by using it in a `for` loop and printing out the images and labels.

In [ ]:
for batch_num, (images, labels) in enumerate(dataloader): 
    print("batch_num:", batch_num)
    print(images)
    print(labels)
    print() ## print a blank line to separate each batch

# BAM!

At long last, we have created the **DataLoader** that we will use to train a convolutional neural network. Now, let's build it!

----

<a id="build"></a>
# Building a convolutional neural network with PyTorch and Lightning

Building a convolutional neural network with PyTorch means creating a new class. And to make it easy to train the neural network, this class will inherit from `LightningModule`.

Our new class will have the following methods:
- `__init__()` to initialize the weights and biases and keep track of a few other housekeeping things.
- `forward()` to make a forward pass through the convolutional neural network.
- `configure_optimizers()` to configure the optimizer. In this tutorial, we'll use `Adam`.
- `training_step()` to pass the training data to `forward()`, calculate the loss, and keep track of the loss values in a log file.

Here is a picture of the neural network we want to create:

<img src="./images/annotated_cnn.png" alt="a simple convolutional neural network" style="width: 600px;">

As we can see in the picture, our convolutional neural network accepts a 6x6 image as the input. Then, it applies a 3x3 filter, or kernel, to the image. That is the convolutional step in a convolutional neural network. Anyway, then we run the output from the filter through a **[ReLU](https://youtu.be/68BZ5f7P94E)** activation function before a max pooling step. Lastly, we run the max pooling values through a simple neural network with 4 inputs and 2 outputs.

**NOTE:** This specific convolutional neural network is super simple. Usually, the images have multiple color channels (one for red, one for blue, and one for green), and multiple filters are applied to each channel. The output from these filters can then be run through multiple layers of filters. In contrast, our example is super simple, so we'll call it `SimpleCNN`.

In [ ]:
## Now build a simple CNN...
class SimpleCNN(L.LightningModule):
    
    def __init__(self):
        
        super().__init__() ## We call the __init__() for the parent, LightningModule, so that it
                           ## can initialize itself as well.
        
        ## Now we set the seed for the random number generorator.
        ## This ensures that when you create a model from this class, that model
        ## will start off with the exact same random numbers that I started out with when
        ## I created this demo. At least, I hope that is what happens!!! :)
        L.seed_everything(seed=42)
        
        ############################################################################
        ##
        ## Here is where we initialize the Weights and Biases for the CNN
        ##
        ############################################################################
        
        ## The filter is created and applied by nn.Conv2d().
        ## in_channels - The number of color channels that
        ##    the image has. Our black and white image only 
        ##    has one channel. However, color pictures usually have 3.
        ## out_channels - If we had multiple input channels, we could merge
        ##    them down to one output. Or we can increase the number of
        ##    output channels if we want.
        ## kernel_size - The size of the filter (aka kernel). In this case
        ##    we want a 3x3 filter, but you can select all kinds of sizes,
        ##    including sizes that are more rectangular than square.
        self.conv = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=3)
        
        ## nn.MaxPool2d() does the max pooling step.
        ## kernel_size - The size of the filter (aka kernel) that does the
        ##    max pooling. We're using a 2x2 grid for our filter.
        ## stride - How much to move the filter each step. In this case
        ##    we're moving it 2 units. Thus, our 2x2 filter does max pooling
        ##    before moving 2 units over (or down). This means that our 
        ##    max pooling filter never overlaps itself.
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        ## Lastly, we create the "normal" neural network that has
        ## 4 inputs, in_features=4, going to a single activation function, out_features=1, 
        ## in a single hidden layer...
        self.input_to_hidden = nn.Linear(in_features=4, out_features=1)
        ## ..and the single hidden layer, in_features=1, goes to
        ## two outputs, out_features=2
        self.hidden_to_output = nn.Linear(in_features=1, out_features=2)
        
        ## We'll use Cross Entropy to calculate the loss between what the 
        ## neural network's predictions and actual, or known, species for
        ## each row in the dataset.
        ## To learn more about Cross Entropy, see: https://youtu.be/6ArSys5qHAU
        ## NOTE: nn.CrossEntropyLoss applies a SoftMax function to the values
        ## we give it, so we don't have to do that oursevles. However,
        ## when we use this neural network (after it has been trained), we'll
        ## have to remember to apply a SoftMax function to the output.
        self.loss = nn.CrossEntropyLoss()

        
    def forward(self, x):
        
        ## First we apply a filter to the input image
        x = self.conv(x)
        
        ## Then we run the output from the filter through a ReLU...
        x = F.relu(x)
        
        ## Then we run the output from the ReLU through a Max Pooling layer...
        x = self.pool(x)
        
        ## Now, at this point we have a square matrix of values.
        ## So, in order to use those values as inputs to
        ## a neural network, we use torch.flatten() to 
        ## turn the matrix into a vector.
        x = torch.flatten(x, 1) # flatten all dimensions except batch 

        ## Now we run the flattened values through a neural network
        ## with a single hidden layer and a single ReLU activation
        ## function in that layer.
        x = self.input_to_hidden(x)
        x = F.relu(x)
        x = self.hidden_to_output(x)
        
        return x

    
    def configure_optimizers(self):
        ## In this example, configuring the optimizer
        ## consists of passing it the weights and biases we want
        ## to optimize, which are all in self.parameters(),
        ## and setting the learning rate with lr=0.001.
        return Adam(self.parameters(), lr=0.001)

    
    def training_step(self, batch, batch_idx):
        ## The first thing we do is split 'batch'
        ## into the input and label values.
        inputs, labels = batch 
        
        ## Then we run the input through the neural network
        outputs = self.forward(inputs)
        
        ## Then we calculate the loss.
        loss = self.loss(outputs, labels)
        
        ## Lastly, we could add the loss to a log file
        ## so that we can graph it later. This would
        ## help us decide if we have done enough training
        ## Ideally, if we do enough training, the loss
        ## should be small and not getting any smaller.
        # self.log("loss", loss) 
        
        return loss

Now that we've created a class for our neural network, let's train it.

----

<a id="train"></a>
# Training our Neural Network

Training our new convolutional neural network means we create a model from the new class, `SimpleCNN`...

In [ ]:
model = SimpleCNN()

...and then create a **Lightning Trainer**, `L.Trainer()`, and use it to optimize the parameters. **NOTE:** We will start with 100 epochs, 100 complete runs through our training data. This may be enough to successfully optimize all of the parameters, but it might not.

In [ ]:
trainer = L.Trainer(max_epochs=100)
trainer.fit(model, train_dataloaders=dataloader)

Hooray! We've trained the model with 100 epochs! Now, let's see if the predictions are any good. We can do this by seeing how well it classifies the original training data. And we can do this using a `for` loop, just like we did earlier when we used to print out the contents of the **DataLoader** that we used for training.

**NOTE:** The **[CrossEntropy](https://youtu.be/6ArSys5qHAU)** loss function we use during training applies a SoftMax function to the outputs. Thus, now that we're using the model, we'll apply `torch.softmax()` to its output. This will make it easier to read and interpret. And to make it even easier to read, we can run it through `torch.round()`.

In [ ]:
for batch_num, (image, label) in enumerate(dataloader):
    
    ## First, run the image through the model to make a prediction
    prediction = model(image)
    
    ## Now make the prediction easy to read and interpret by
    ## running it through torch.softmax() and torch.round()
    predicted_label = torch.round(torch.softmax(prediction, dim=1), ## dim=0 applies softmax to rows, dim=1 applies soft to columns
                                  decimals=2) 
    
    ## Now print out the the predicted label and the original label
    ## so we see how well our CNN performed.
    print("predicted_label:", predicted_label)
    print("original label:", label)
    print("\n")

As we can see in the output printed above, after 100 epochs of training, our convolutional neural network is not making good predictions. So let's do a lot more training.

The good news is that because we're using **Lightning**, we can pick up where we left off training without starting over from scratch. This is because training with **Lightning** creates _checkpoint_ files that keep track of the Weights and Biases as they change. As a result, all we have to do to pick up where we left off is tell the `Trainer` where the checkpoint files are. This is awesome and will save us a lot of time since we don't have to retrain the first **100** epochs. So, let's add an additional **600** epochs to the training.

To add additional epochs to the training, we first identify where the checkpoint file is with the following command.

In [ ]:
path_to_checkpoint = trainer.checkpoint_callback.best_model_path ## By default, "best" = "most recent"

Then we create a new **Lightning Trainer**, just like before, but we set the number of epochs to 700. Given that we already trained for 100 epochs, this means we'll do 600 more.

In [ ]:
## First, create a new Lightning Trainer
trainer = L.Trainer(max_epochs=700) # Before, max_epochs=100, so, by setting it to 700, we're adding 600 more.

## Then call trainer.fit() using the path to the most recent checkpoint files
## so that we can pick up where we left off.
trainer.fit(model, train_dataloaders=dataloader, ckpt_path=path_to_checkpoint)

Now, let's see if the predictions have improved after 700 epochs. We'll do this using the same `for` loop that we used before.

In [ ]:
for batch_num, (image, label) in enumerate(dataloader):
    
    ## First, run the image through the model to make a prediction
    prediction = model(image)
    
    ## Now make the prediction easy to read and interpret by
    ## running it through torch.softmax() and torch.round()
    predicted_label = torch.round(torch.softmax(prediction, dim=1), ## dim=0 applies softmax to rows, dim=1 applies soft to columns
                                  decimals=2) 
    
    ## Now print out the the predicted label and the original label
    ## so we see how well our CNN performed.
    print("predicted_label:", predicted_label)
    print("original label:", label)
    print("\n")

After 700 training epochs, the predicted values are much, much closer to the ideal values. And if we rounded them to the nearest whole number, we'd get the exact labels that we wanted.

# Double BAM!!

----

<a id="predict"></a>
# Make a Prediction with New Data

Now that our model is trained, we can use it to make predictions from new data. Specifically, let's see how well our model predicts an image of the letter **X** that has been shifted over one pixel. First, let's create the image of an **X** shifted by one pixel to the right.

In [ ]:
shifted_x_image = [[0, 1, 0, 0, 0, 0],
                   [0, 0, 1, 0, 0, 1],
                   [0, 0, 0, 1, 1, 0],
                   [0, 0, 0, 1, 1, 0],
                   [0, 0, 1, 0, 0, 1],
                   [0, 1, 0, 0, 0, 0]]
shifted_x_image

Now, let's verify that we created the correct image by drawing with matplotlib, just like we did earlier.

In [ ]:
## When we only want to draw one image, we can omit 'nrows' and 'ncols' and 
## a single drawing element, ax, is returned.
fig, ax = plt.subplots(figsize=(2.5, 2.5))
ax.imshow(shifted_x_image, cmap='gray_r') ## Setting cmap='gray_r' gives us reverse grayscale.

Now that we can see that we correctly created an image of an **X** shifted one pixel to the right, let's see whether or not our trained convolutional neural network can correctly classify it as an **X**.

In [ ]:
## First, let's make a prediction with the new image...
prediction = model(torch.tensor([shifted_x_image]).type(torch.float32))

## Now make the prediction easy to read and interpret by
## running it through torch.softmax() and torch.round()
predicted_label = torch.round(torch.softmax(prediction, dim=1), decimals=2) ## dim=0 applies argmax to rows, dim=1 applies argmax to colum

predicted_label

And we see that the trained network correctly predicted **X** since the second output value, which represents **X**, is larger than the first output value, which represents **O**.

Now, let's see if our trained network can correctly classify an image of the letter **O** that is shifted one pixel to the left. We'll do this just like we did for the shifted **X**. So, the first thing we do is create the image...

In [ ]:
shifted_o_image = [[0, 1, 1, 0, 0, 0],
                   [1, 0, 0, 1, 0, 0],
                   [0, 0, 0, 0, 1, 0],
                   [0, 0, 0, 0, 1, 0],
                   [1, 0, 0, 1, 0, 0],
                   [0, 1, 1, 0, 0, 0]]
shifted_o_image

...then we display the image to verify that we made it correctly...

In [ ]:
## When we only want to draw one image, we can omit 'nrows' and 'ncols' and 
## a single drawing element, ax, is returned.
fig, ax = plt.subplots(figsize=(2.5, 2.5))
ax.imshow(shifted_o_image, cmap='gray_r') ## Setting cmap='gray_r' gives us reverse grayscale.

Lastly, we run the image through the trained network to make a prediction...

In [ ]:
## Make a prediction with the new image...
prediction = model(torch.tensor([shifted_o_image]).type(torch.float32))

## Now make the prediction easy to read and interpret by
## running it through torch.softmax() and torch.round()
predicted_label = torch.round(torch.softmax(prediction, dim=1), decimals=2) ## dim=0 applies argmax to rows, dim=1 applies argmax to colum

predicted_label

And it looks like our network thought that the shifted **O** was an **X**. Oh well.

Small bam.

However, we learned a lot, and with more training data and more variation in the letters, we might be able to make better predictions in the future!

# TRIPLE BAM!!!